In [1]:
import requests, json, os
import pandas as pd
import numpy as np
import time
import sqlite3
import warnings

from pandas.io.json import json_normalize
from sqlalchemy import create_engine, inspect
from datetime import datetime
from TDA_Functions import *
from config import client_id

In [2]:
# gain the ability to see all rows/columns if desired
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# ignore warnings
warnings.filterwarnings("ignore")  

In [3]:
# create the stocks sql database
engineStocksPredict = create_engine("sqlite:///stocks_predict.db", echo = False)
# connect to the stocks sql database
connStocksPredict = sqlite3.connect('stocks_predict.db')
# create the ability to perform logic on the database 
s = connStocksPredict.cursor()
# create the stock table in the stocks database
s.execute('CREATE TABLE IF NOT EXISTS stocks_table '\
          '(ticker TEXT, year INTEGER, quarter INTEGER, yearStuck INTEGER, yearGreen INTEGER, longYrStuck INTEGER, longCompndYrStuck INTEGER, '\
          'pivotGreenYear INTEGER, quarterStuck INTEGER, quarterGreen INTEGER, pivotGreenQuarter INTEGER, longQtrStuck INTEGER, longCompndQtrStuck INTEGER, '\
          'revStratUp TEXT, mgbuy INTEGER, timeBreached INTEGER, '\
          'threeMoLow INTEGER, sixMoLow INTEGER, threeMoHigh INTEGER, sixMoHigh INTEGER, '\
          'stuckInTheMiddle INTEGER, cmpndStuck INTEGER, stuck INTEGER, combo TEXT, '\
          'month TEXT, dateBreachedF1 TEXT, perCapture INTEGER, perRisk INTEGER, '\
          'riskReward INTEGER, threeMoChng INTEGER, target TEXT, '\
          'PRIMARY KEY (ticker, year, quarter, yearStuck, yearGreen, pivotGreenYear, longYrStuck, longCompndYrStuck, quarterStuck, '\
          'quarterGreen, pivotGreenQuarter, longQtrStuck, longCompndQtrStuck,revStratUp, mgbuy, timeBreached, threeMoLow, sixMoLow, '\
          'threeMoHigh, sixMoHigh, stuckInTheMiddle, cmpndStuck, stuck, combo, month, '\
          'dateBreachedF1, perCapture, perRisk, riskReward, threeMoChng, target))')
# commit the stock table
connStocksPredict.commit()
# check the stock table and make sure it was correctly added to the database
print(inspect(engineStocksPredict).get_table_names())

['stocks_table']


In [4]:
# # print all column names without quotes
# print(*month_df.columns, sep=', ')

In [5]:
csv_df = pd.read_csv('StockList.csv')
stock_list = csv_df['Symbol'].tolist()
test_list = ['XONE']

In [6]:
# create a count to keep track of stocks accessed
count = 0
# time how long it takes for the code to run
start_time = time.time()
print(datetime.now())

for i in stock_list:
    ticker = i
    print(ticker)
    try:
        # define our endpoint
        endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(ticker)
        # define our payload
        payload = {'apikey':client_id,
                   'periodType':'year',
                   'period':'5',
                   'frequencyType':'daily',
                   'frequency':'1',
                   'needExtendedHoursData':'false'}
        # make a request
        content = requests.get(url = endpoint, params = payload)
        # convert it to a dictionary using JSON method
        data = content.json()
        day_df = pd.json_normalize(data['candles'])
        day_df['datetime'] = pd.to_datetime(day_df['datetime'], unit='ms')
        day_df['full_date'] = pd.to_datetime(day_df['datetime']).dt.date
        # create monthly identifier column 
        day_df['month'] = pd.to_datetime(day_df['datetime'], format="%m%Y").dt.to_period('m')
        day_df['month'] = day_df['month'].astype(str)
        # create quarterly identifier column 
        day_df['quarter'] = quarterMaker(day_df['month'])
        # create yearly identifier column 
        day_df['year'] = yearMaker(day_df['month'])
        # drop the volume quarter
        day_df.drop('volume', axis=1, inplace=True)
        # create a useable df
        day_df = day_df.set_index('datetime')
        # create a quarterly df from the daily data that produces each quarter's open, high, low and close
        agg_dict = {'open': 'first',
                  'high': 'max',
                  'low': 'min',
                  'close': 'last',
                  'month': 'max',  
                  'quarter': 'max'}
        temp_quarter_df = day_df.resample('Q', convention='end').agg(agg_dict)
        # shifting columns for ability to do calcs on rows in the past
        temp_quarter_df = temp_quarter_df.rename(columns={'open':'openQuarterly'})
        temp_quarter_df['longQtrStuck'] = stuckIndicator(temp_quarter_df['high'], temp_quarter_df['low'])
        temp_quarter_df['longCompndQtrStuck'] = compoundStuckIndicator(temp_quarter_df['high'], temp_quarter_df['low'])
        temp_quarter_df['high1Quarterly'] = temp_quarter_df['high'].shift(periods=1)
        temp_quarter_df['low1Quarterly'] = temp_quarter_df['low'].shift(periods=1)
        # create df of columns from temp_quarter_df needed for temp_df merge
        abr_quarter_df = temp_quarter_df[['openQuarterly', 'high1Quarterly', 'low1Quarterly', 'quarter', 'longQtrStuck', 'longCompndQtrStuck']]
        # create a yearly df from the daily data that produces each year's open, high, low and close
        agg_dict = {'open': 'first',
                  'high': 'max',
                  'low': 'min',
                  'close': 'last',
                  'year': 'max'}
        temp_year_df = day_df.resample('Y').agg(agg_dict)
        # shifting columns for ability to do calcs on rows in the past
        temp_year_df = temp_year_df.rename(columns={'open':'openYearly'})
        temp_year_df['longYrStuck'] = stuckIndicator(temp_year_df['high'], temp_year_df['low'])
        temp_year_df['longCompndYrStuck'] = compoundStuckIndicator(temp_year_df['high'], temp_year_df['low'])
        temp_year_df['high1Yearly'] = temp_year_df['high'].shift(periods=1)
        temp_year_df['low1Yearly'] = temp_year_df['low'].shift(periods=1)
        # create df of columns from temp_year_df needed for temp_df merge
        abr_year_df = temp_year_df[['openYearly', 'high1Yearly', 'low1Yearly', 'year', 'longYrStuck', 'longCompndYrStuck']]
        # create a monthly df from the daily data that produces each month's open, high, low and close
        agg_dict = {'open': 'first',
                  'high': 'max',
                  'low': 'min',
                  'close': 'last',
                  'month': 'max'}
        temp_month_df = day_df.resample('M').agg(agg_dict)
        # shifting columns for ability to do calcs on rows in the past
        temp_month_df = temp_month_df.rename(columns={'high':'highMonthly'})
        temp_month_df['high1'] = temp_month_df['highMonthly'].shift(periods=1)
        temp_month_df['low1'] = temp_month_df['low'].shift(periods=1)
        # create df of columns from temp_month_df needed for temp_df merge
        abr_month_df = temp_month_df[['highMonthly', 'high1', 'low1', 'month']]
        # create new temp_df to get dayCheck desired columns
        temp_df = pd.merge(day_df, abr_month_df, on='month', how='outer')
        temp_df = pd.merge(temp_df, abr_quarter_df, on='quarter', how='outer')
        temp_df = pd.merge(temp_df, abr_year_df, on='year', how='outer')
        # make sure full_date is in str format to extrat dates
        temp_df['full_date'] = temp_df['full_date'].astype(str)
        # remove all nan's, if they exist, from the dataframe so the dataframe can be operated on
        temp_df = temp_df[temp_df['full_date'] != "nan"]
        # create day column to use in the dayCheck helper function
        temp_df['day'] = temp_df['full_date'].str[-2:]
        temp_df['day'] = temp_df['day'].astype(int)
        # run the dayCheck helper function to get needed tuples for final df
        temp_df['helper'] = dayCheck(temp_df['day'], temp_df['low'], temp_df['high'], temp_df['open'], temp_df['high1'], temp_df['low1'], temp_df['full_date'])
        temp_df['quarterHelper'] = quarterCheck(temp_df['low'], temp_df['high'], temp_df['close'], temp_df['openQuarterly'], temp_df['low1Quarterly'], temp_df['high1Quarterly'], temp_df['highMonthly'], temp_df['quarter'])
        temp_df['yearHelper'] = yearCheck(temp_df['low'], temp_df['high'], temp_df['close'], temp_df['openYearly'], temp_df['low1Yearly'], temp_df['high1Yearly'], temp_df['highMonthly'], temp_df['year'])
        # rename columns from the tuple helper column
        col_list = ['monthLow','breached','gapped','failed','periodBreached','topBottom', 'dateBreached']
        for i, col in enumerate(col_list):
            temp_df[col] = temp_df['helper'].apply(lambda helper: helper[i])
        temp_df = temp_df.drop('helper',axis=1)
        # rename columns from the tuple quarterHelper column
        col_list = ['quarterStuck','quarterGreen', 'pivotGreenQuarter']
        for i, col in enumerate(col_list):
            temp_df[col] = temp_df['quarterHelper'].apply(lambda helper: helper[i])
        temp_df = temp_df.drop('quarterHelper',axis=1)
        # rename columns from the tuple yearHelper column
        col_list = ['yearStuck','yearGreen', 'pivotGreenYear']
        for i, col in enumerate(col_list):
            temp_df[col] = temp_df['yearHelper'].apply(lambda helper: helper[i])
        temp_df = temp_df.drop('yearHelper',axis=1)
        # transform year and qurter columns for feature engineering
        temp_df['year'] = temp_df['year'].astype(str)
        temp_df['quarter'] = temp_df['quarter'].apply(lambda x: int(x.split('-')[1]))
        # create a column that identifies if the row is the last day in the month or not
        temp_df['endOfMonth'] = endOfMonth(temp_df['day'])
        # clearn temp_df so it is ready to merge with temp_month_df
        temp_df = temp_df[temp_df['endOfMonth'] == 1]
        temp_df.reset_index(drop=True, inplace=True)
        temp_df = temp_df[['month', 'day', 'monthLow', 'breached', 'gapped', 'failed', 'periodBreached', 'topBottom', 'dateBreached', 'quarterStuck', 'longQtrStuck', 'longCompndQtrStuck', \
                           'year', 'quarter', 'quarterGreen', 'pivotGreenQuarter', 'yearStuck','yearGreen', 'pivotGreenYear', 'longYrStuck', 'longCompndYrStuck']]
        # return temp_month_df column back to regular name 
        temp_month_df = temp_month_df.rename(columns={'highMonthly':'high'})
        # create new df with day and month data how we want it
        month_df = pd.merge(temp_df, temp_month_df, on='month', how='outer')
        #drop the high1 column so not doubled up in final month_df
        month_df = month_df.drop('high1',axis=1)
        # insert ticker name into df
        month_df.insert(0, 'ticker', ticker)

        # shifting columns for ability to do calcs on rows in the past
        month_df['high1'] = month_df['high'].shift(periods=1)
        month_df['high2'] = month_df['high'].shift(periods=2)
        month_df['high3'] = month_df['high'].shift(periods=3)
        month_df['high4'] = month_df['high'].shift(periods=4)
        month_df['high5'] = month_df['high'].shift(periods=5)
        month_df['high6'] = month_df['high'].shift(periods=6)
        month_df['low1'] = month_df['low'].shift(periods=1)
        month_df['low2'] = month_df['low'].shift(periods=2)
        month_df['low3'] = month_df['low'].shift(periods=3)
        month_df['low4'] = month_df['low'].shift(periods=4)
        month_df['low5'] = month_df['low'].shift(periods=5)
        month_df['low6'] = month_df['low'].shift(periods=6)
        month_df['close1'] = month_df['close'].shift(periods=1)
        month_df['open1'] = month_df['open'].shift(periods=1)
        month_df['diff'] = month_df['high'] - month_df['low']
        month_df['diff1'] =  month_df['diff'].shift(periods=1)
        month_df['top33'] = month_df['high'] - (month_df['diff'] * .34)
        month_df['prevTop33'] = month_df['top33'].shift(periods=1)
        month_df['bot33'] = month_df['low'] + (month_df['diff'] * .34)
        month_df['prevBot33'] = month_df['bot33'].shift(periods=1)
        # High/Low/Close/Open/Volume in the past
        HI = month_df['high']
        HI1 = month_df['high1']
        HI2 = month_df['high2']
        HI3 = month_df['high3']
        HI4 = month_df['high4']
        HI5 = month_df['high5']
        HI6 = month_df['high6']
        LO = month_df['low']
        LO1 = month_df['low1']
        LO2 = month_df['low2']
        LO3 = month_df['low3']
        LO4 = month_df['low4']
        LO5 = month_df['low5']
        LO6 = month_df['low6']
        OP = month_df['open']
        OP1 = month_df['open1']
        CL = month_df['close']
        CL1 = month_df['close1']
        # Close >= Open
        CL_OP = (CL >= OP)
        # Difference and top/bot 33% name assignments
        DIF = month_df['diff']
        DIF1 = month_df['diff1']
        TOP33 = month_df['top33']
        TOP331 = month_df['prevTop33']
        BOT33 = month_df['bot33']
        BOT331 = month_df['prevBot33']
        #check if bar is a hammer or shooter
        HamCheck = (OP >= TOP33) & (CL >= TOP33)
        ShootCheck = (OP <= BOT33) & (CL < BOT33)
        #Inside Bars
        Inside = (HI <= HI1) & (LO >= LO1)
        InsideOne = (HI1 <= HI2) & (LO1 >= LO2)
        InsideTwo = (HI2 <= HI3) & (LO2 >= LO3)
        #Outside Bar
        Outside = (HI > HI1) & (LO < LO1)
        OutsideOne = (HI1 > HI2) & (LO1 < LO2)
        OutsideTwo = (HI2 > HI3) & (LO2 < LO3)
        OutsideThree = (HI3 > HI4) & (LO3 < LO4)
        #Soft Setup
        SoftSetupUp =  (CL <= HI1) & (CL >= LO1) & (OP <= HI1) & (OP >= LO1) & (CL > BOT33)
        SoftSetupUp1 = (CL1 <= HI2) & (CL1 >= LO2) & (OP1 <= HI2) & (OP1 >= LO2) & (CL1 > BOT331)
        SoftSetupDn =  (CL <= HI1) & (CL >= LO1) & (OP <= HI1) & (OP >= LO1) & (CL < TOP33)
        SoftSetupDn1 = (CL1 <= HI2) & (CL1 >= LO2) & (OP1 <= HI2) & (OP1 >= LO2) & (CL1 < TOP331)
        #Two Bar
        TwoUp = (OP >= TOP33) & (CL >= TOP33) & (LO < LO1) & (HI <= HI1)
        TwoDn = (OP <= BOT33) & (CL <= BOT33) & (LO >= LO1) & (HI > HI1)
        TwoUp1 = (OP1 >= TOP331) & (CL1 >= TOP331) & (LO1 < LO2) & (HI1 <= HI2)
        TwoDn1 = (OP1 <= BOT331) & (CL1 <= BOT331) & (LO1 >= LO2) & (HI1 > HI2)
        SoftTwoUp = (LO < LO1) & (HI <= HI1) & SoftSetupUp
        SoftTwoDn = (LO >= LO1) & (HI > HI1) & SoftSetupDn
        SoftTwoUp1 = (LO1 < LO2) & (HI1 <= HI2) & SoftSetupUp1
        SoftTwoDn1 = (LO1 >= LO2) & (HI1 > HI2) & SoftSetupDn1
        ##Rev Strat
        #122, bullish/bearish 2 bar rev strat
        InsideAndUp = (InsideOne & (TwoUp | (SoftTwoUp & (LO <= LO2))))
        InsideAndDn = (InsideOne & (TwoDn | (SoftTwoDn & (LO <= LO2))))
        #13, bullish/bearish 1 bar rev strat
        month_df['revStratUp'] = (InsideOne & Outside & (CL > OP)).astype(int)          # OneBarRevStratUp
        # month_df['RevStratDn'] = (InsideOne & Outside & (CL < OP)).astype(int)        # OneBarRevStratDn
        revStratUp = (InsideOne & Outside & (CL > OP)).astype(int)
        revStratDn = (InsideOne & Outside & (CL < OP)).astype(int) 
        ## Other bar types
        # Inside/Outside/Inside
        InsideOutsideInsideUp = (InsideTwo & OutsideOne & Inside)
        #227
        DoubleHam = (CL1 >= TOP331) & (OP1 >= TOP331) & HamCheck & (HI2 > HI1) & (HI2 > HI)
        DoubleShoot = (CL1 <= BOT331) & (OP1 <= BOT331) & ShootCheck
        #312 old formula
        # ThreeOneTwoUp = (OutsideTwo & InsideOne & (TwoUp | (SoftTwoUp & (LO <= LO2))))
        # ThreeOneTwoDn = (OutsideTwo & InsideOne & (TwoUp | (SoftTwoUp & (LO <= LO2))))
        #312
        ThreeOneTwoUp = (OutsideTwo & InsideOne & (TwoUp | SoftTwoUp))
        ThreeOneTwoDn = (OutsideTwo & InsideOne & (TwoUp | SoftTwoUp))
        #31
        OutIn = (OutsideOne & Inside)
        #322
        OutsideAndUp = (OutsideOne & (SoftTwoUp | TwoUp))
        OutsideAndDn = (OutsideOne & (SoftTwoDn | TwoDn))
        #727
        Two2Up = (TwoDn1 | SoftTwoDn1) & (TwoUp | SoftTwoUp)
        Two2Dn = (TwoUp1 | SoftTwoUp1) & (TwoDn | SoftTwoDn1)
        ### Pivot Machine Gun Month, old formula ("PMG")
        # month_df['mgbuy'] = ((HI <= HI1) & (HI1 <= HI2) & (Inside | (CL >= LO1) | TwoUp)).astype(int)
        # month_df['mgsale'] = ((LO >= LO1) & (LO1 >= LO2) & (Inside | (CL <= HI1) | TwoDn)).astype(int)
        ### PMG new formula
        month_df['mgbuy'] = (((HI <= HI1) & (HI1 <= HI2) & (HI2 <= HI3))).astype(int)
        # PMG Setup
        PMG = ((LO < LO1) & (HI <= HI1) & (HI1 <= HI2) & (HI2 <= HI3))
        ### Monthly Low Indicator
        month_df['threeMoLow'] = find3MoLow(month_df['low'])
        month_df['sixMoLow'] = find6MoLow(month_df['low'])
        ### Monthly Hi Indicator
        month_df['threeMoHigh'] = find3MoHi(month_df['high'])
        month_df['sixMoHigh'] = find6MoHi(month_df['high'])
        ### Gap Indicators
        # monthly
        month_df['gap'] = (OP > HI1).astype(int)
        month_df['gap1'] = month_df['gap'].shift(periods=-1)
        # daily gap over prev month's high
        month_df['dayGap1'] = month_df['gapped'].shift(periods=-1)
        # create stuck indicator column, is the bar stuck in the previous bar's high and low
        month_df['stuck'] = stuckIndicator(month_df['high'], month_df['low'])
        # create compound stuck indicator column, is the bar stuck in the previous two bar high and low (compound outside)
        month_df['cmpndStuck'] = compoundStuckIndicator(month_df['high'], month_df['low'])
        # calc the different set ups for execution using cominations of the definitions from above
        comboConditions = [
            (ThreeOneTwoUp == 1), 
            (OutIn == 1),
            ((Inside == 1) & (InsideOne == 1) & (InsideTwo == 1)) ,
            ((Inside == 1) & (InsideOne == 1) & (InsideTwo == 0)),
            ((Inside == 1) & (InsideOne == 0) & ((InsideTwo == 0) | (InsideTwo == 1))),
            (OutsideAndUp == 1), 
            (DoubleHam == 1), 
            (Two2Up == 1), 
            (InsideAndUp == 1),
            (PMG == 1)]
        comboChoices = [
            "312U",
            "31",
            "111",
            "11",
            "1",
            "32U",
            "2xHam", 
            "22U",
            "12U",
            "PMG"]
        month_df['combo'] = np.select(comboConditions, comboChoices, default='0')

        # shifting columns for ability to do calcs on rows in the future
        month_df['highF1'] = month_df['high'].shift(periods=-1)
        month_df['highF2'] = month_df['high'].shift(periods=-2)
        month_df['highF3'] = month_df['high'].shift(periods=-3)
        month_df['lowF1'] = month_df['low'].shift(periods=-1)
        month_df['lowF2'] = month_df['low'].shift(periods=-2)
        month_df['lowF3'] = month_df['low'].shift(periods=-3)
        month_df['openF2'] = month_df['open'].shift(periods=-2)
        month_df['openF3'] = month_df['open'].shift(periods=-3)
        month_df['closeF2'] = month_df['close'].shift(periods=-2)
        month_df['closeF3'] = month_df['close'].shift(periods=-3)
        month_df['monthLowF1'] = month_df['monthLow'].shift(periods=-1)
        month_df['failedF1'] = month_df['failed'].shift(periods=-1)
        month_df['periodBreachedF1'] = month_df['periodBreached'].shift(periods=-1)
        month_df['dateBreachedF1'] = month_df['dateBreached'].shift(periods=-1)
        # assign columns to variable names for readability
        HIF1 = month_df['highF1']
        HIF2 = month_df['highF2']
        HIF3 = month_df['highF3']
        LOF1 = month_df['lowF1']
        LOF2 = month_df['lowF2']
        LOF3 = month_df['lowF3']
        COMBO = month_df['combo']
        GAP1 = month_df['gap1']
        DAYGAP1 = month_df['dayGap1']
        OPF2 = month_df['openF2']
        OPF3 = month_df['openF3']
        CLF2 = month_df['closeF2']
        CLF3 = month_df['closeF3']
        RISKLOWF1 = month_df['monthLowF1']
        FAILED1 = month_df['failedF1']

        # add percent risk to capture column
        month_df['perRisk'] = round(((HI - RISKLOWF1)/abs(HI))*100, 1)
        # add column that calcs the three month percent change if position was kept on for that duration
        month_df['threeMoChng']  = round(((CLF3 - HI)/abs(HI))*100, 1)
        # booleans to determine if a stock reaches its target, makes no attempt or fails for all setups and their respective target lookback,
        # be it (one period, two periods or three periods)
        TARGET3 = ((HIF1 > HI) & (FAILED1 == 0) & \
                ((GAP1 == 0) & (DAYGAP1 == 0)) & \
                (COMBO == '111') & \
                ((HIF1 >= HI3) | ((HIF2 >= HI3) & (LOF2 >= RISKLOWF1)) | ((HIF3 >= HI3) & (LOF3 >= RISKLOWF1) & (LOF2 >= RISKLOWF1))))

        TARGET2 = ((HIF1 > HI) & (FAILED1 == 0) & \
                ((GAP1 == 0) & (DAYGAP1 == 0)) & \
                ((COMBO == '11') | (COMBO == '312U') | (COMBO == '2xHam') | (COMBO == '12U')) & \
                ((HIF1 >= HI2) | ((HIF2 >= HI2) & (LOF2 >= RISKLOWF1)) | ((HIF3 >= HI2) & (LOF3 >= RISKLOWF1) & (LOF2 >= RISKLOWF1))))

        TARGET1 = ((HIF1 > HI) & (FAILED1 == 0) & \
                ((GAP1 == 0) & (DAYGAP1 == 0)) & \
                ((COMBO == '31') | (COMBO == '32U') | (COMBO == '22U') | (COMBO == 'PMG') | (COMBO == '1')) & \
                ((HIF1 >= HI1) | ((HIF2 >= HI1) & (LOF2 >= RISKLOWF1)) | ((HIF3 >= HI1) & (LOF3 >= RISKLOWF1) & (LOF2 >= RISKLOWF1))))

        NOATTEMPT3 = ((((GAP1 == 1) | (DAYGAP1 == 1)) | (HIF1 <= HI)) & \
                (COMBO == '111'))

        NOATTEMPT2 = ((((GAP1 == 1) | (DAYGAP1 == 1)) | (HIF1 <= HI)) & \
                ((COMBO == '11') | (COMBO == '312U') | (COMBO == '2xHam') | (COMBO == '12U')))

        NOATTEMPT1 = ((((GAP1 == 1) | (DAYGAP1 == 1)) | (HIF1 <= HI)) & \
                ((COMBO == '31') | (COMBO == '32U') | (COMBO == '22U') | (COMBO == 'PMG') | (COMBO == '1')))

        FAIL3 = (((HIF1 > HI) & \
                ((GAP1 == 0) & (DAYGAP1 == 0)) & \
                ((FAILED1 == 1) | ((LOF2 < RISKLOWF1) | (LOF3 < RISKLOWF1))) & \
                (COMBO == '111')))

        FAIL2 = (((HIF1 > HI) & \
                ((GAP1 == 0) & (DAYGAP1 == 0)) & \
                ((FAILED1 == 1) | ((LOF2 < RISKLOWF1) | (LOF3 < RISKLOWF1))) & \
                ((COMBO == '11') | (COMBO == '312U') | (COMBO == '2xHam') | (COMBO == '12U'))))


        FAIL1 = (((HIF1 > HI) & \
                ((GAP1 == 0) & (DAYGAP1 == 0)) & \
                ((FAILED1 == 1) | ((LOF2 < RISKLOWF1) | (LOF3 < RISKLOWF1))) & \
                ((COMBO == '31') | (COMBO == '32U') | (COMBO == '22U') | (COMBO == 'PMG') | (COMBO == '1'))))

        STILLWORKING3 = (((HIF3 < HI3) | (HIF2 < HI3) | (HIF1 < HI3)) & (COMBO == '111'))

        STILLWORKING2 = (((HIF3 < HI2) | (HIF2 < HI2) | (HIF1 < HI2)) & ((COMBO == '11') | (COMBO == '312U') | (COMBO == '2xHam') | (COMBO == '12U')))

        STILLWORKING1 = (((HIF3 < HI1) | (HIF2 < HI1) | (HIF1 < HI1)) & ((COMBO == '31') | (COMBO == '32U') | (COMBO == '22U') | (COMBO == 'PMG') | (COMBO == '1')))

        # calc if the bar eventually reached its target, made no attempt, or failed
        targetConditions = [
                    (TARGET3 == 1), 
                    (TARGET2 == 1),
                    (TARGET1 == 1),
                    (NOATTEMPT3 == 1), 
                    (NOATTEMPT2 == 1),
                    (NOATTEMPT1 == 1),
                    (FAIL3 == 1), 
                    (FAIL2 == 1),
                    (FAIL1 == 1),
                    (STILLWORKING3 == 1), 
                    (STILLWORKING2 == 1),
                    (STILLWORKING1 == 1)]
        targetChoices = [
                        'Success',
                        'Success',
                        'Success',
                        'NoAttempt',
                        'NoAttempt',
                        'NoAttempt',
                        'Failure',
                        'Failure',
                        'Failure',
                        'StillWorking',
                        'StillWorking',
                        'StillWorking']
        month_df['target'] = np.select(targetConditions, targetChoices, default='0')

        # calc percent trying to capture (know the percent gaining if traget met)
        PERCENT3 = round(((HI3 - HI)/abs(HI))*100, 1)
        PERCENT2 = round(((HI2 - HI)/abs(HI))*100, 1)
        PERCENT1 = round(((HI1 - HI)/abs(HI))*100, 1)
        PERCENTWORKING3 = round(((HI3 - HI)/abs(HI))*100, 1)/2
        PERCENTWORKING2 = round(((HI2 - HI)/abs(HI))*100, 1)/2
        PERCENTWORKING1 = round(((HI1 - HI)/abs(HI))*100, 1)/2
        percentConditions = [
                    (TARGET3 == 1), 
                    (TARGET2 == 1),
                    (TARGET1 == 1),
                    (NOATTEMPT3 == 1), 
                    (NOATTEMPT2 == 1),
                    (NOATTEMPT1 == 1),
                    (FAIL3 == 1), 
                    (FAIL2 == 1),
                    (FAIL1 == 1),
                    (STILLWORKING3 == 1), 
                    (STILLWORKING2 == 1),
                    (STILLWORKING1 == 1)]
        percentChoices = [
                        PERCENT3,
                        PERCENT2,
                        PERCENT1,
                        PERCENT3,
                        PERCENT2,
                        PERCENT1,
                        PERCENT3,
                        PERCENT2,
                        PERCENT1,
                        PERCENTWORKING3,
                        PERCENTWORKING2,
                        PERCENTWORKING1]
        month_df['perCapture']  = np.select(percentConditions, percentChoices, default=0)

        # calc if a one bar is stuck in the middle
        month_df['stuckInTheMiddle'] = stuckInTheMiddle(month_df['high'], month_df['low'], month_df['combo'])
        # add risk reward ratio column
        month_df['riskReward'] = round(month_df['perCapture'] / month_df['perRisk'], 2)
        # add time breached column
        month_df['timeBreached'] = timeBreached(month_df['periodBreached'], month_df['target'])
        month_df['timeBreached'] = month_df['timeBreached']
        # make sure year column is float
        month_df['year'] = month_df['year'].astype(float)
        # create the final monthly desired form of the df
        month_df = month_df[['ticker', 'year', 'quarter', 'yearStuck','yearGreen', 'pivotGreenYear', 'longYrStuck', 'longCompndYrStuck', \
                             'quarterStuck','quarterGreen', 'pivotGreenQuarter', 'longQtrStuck', 'longCompndQtrStuck', 'revStratUp', \
                             'mgbuy', 'timeBreached', 'threeMoLow', 'sixMoLow', 'threeMoHigh', 'sixMoHigh', 'stuckInTheMiddle', 'cmpndStuck', 'stuck', 'combo', \
                             'month', 'dateBreachedF1', 'perCapture', 'perRisk', 'riskReward', 'threeMoChng', 'target']]
        month_df = month_df[(month_df['combo'] != '0')]
        # add 1 to total count and print count
        count += 1
        # replace database if first pull, otherwise append data to its respective database
        if count == 1:
            month_df.to_sql('stocks_table', connStocksPredict, if_exists='replace', index=False)
        else:
            month_df.to_sql('stocks_table', connStocksPredict, if_exists='append', index=False)
        print(month_df.shape)
        print(count)
    except: print(f"-----Did not pull data for: {ticker}")
    
print("This script took", round((time.time() - start_time)/60, 2), "minutes to run.")

2021-10-28 12:50:14.682104
AAPL
(51, 31)
1
MSFT
(66, 31)
2
GOOG
(53, 31)
3
AMZN
(51, 31)
4
FB
(25, 31)
5
TSLA
(35, 31)
6
TSM
(66, 31)
7
NVDA
(55, 31)
8
JPM
(57, 31)
9
V
(39, 31)
10
BABA
(22, 31)
11
JNJ
(69, 31)
12
UNH
(48, 31)
13
WMT
(65, 31)
14
BAC
(78, 31)
15
HD
(55, 31)
16
MA
(34, 31)
17
PHIG
(23, 31)
18
PG
(47, 31)
19
ASML
(54, 31)
20
DIS
(61, 31)
21
ADBE
(47, 31)
22
NFLX
(54, 31)
23
CRM
(46, 31)
24
PYPL
(13, 31)
25
ORCL
(59, 31)
26
XOM
(56, 31)
27
NKE
(50, 31)
28
NVO
(60, 31)
29
PFE
(78, 31)
30
TMO
(49, 31)
31
TM
(75, 31)
32
KO
(73, 31)
33
LLY
(56, 31)
34
CSCO
(53, 31)
35
ABT
(56, 31)
36
DHR
(47, 31)
37
ACN
(55, 31)
38
PEP
(55, 31)
39
VZ
(76, 31)
40
CVX
(70, 31)
41
COST
(64, 31)
42
AVGO
(27, 31)
43
MRK
(75, 31)
44
WFC
(68, 31)
45
INTC
(53, 31)
46
SE
(8, 31)
47
ABBV
(23, 31)
48
AZN
(52, 31)
49
NVS
(57, 31)
50
TXN
(67, 31)
51
MS
(55, 31)
52
T
(68, 31)
53
MCD
(57, 31)
54
SHOP
(22, 31)
55
UPS
(62, 31)
56
SAP
(51, 31)
57
MDT
(73, 31)
58
NEE
(44, 31)
59
LIN
(51, 31)
60
INTU
(52, 31)
61


In [7]:
stocks = pd.read_sql_query('''SELECT * FROM "stocks_table"''', connStocksPredict)
stocks.tail()

,ticker,year,quarter,yearStuck,yearGreen,pivotGreenYear,longYrStuck,longCompndYrStuck,quarterStuck,quarterGreen,pivotGreenQuarter,longQtrStuck,longCompndQtrStuck,revStratUp,mgbuy,timeBreached,threeMoLow,sixMoLow,threeMoHigh,sixMoHigh,stuckInTheMiddle,cmpndStuck,stuck,combo,month,dateBreachedF1,perCapture,perRisk,riskReward,threeMoChng,target
180263,CDR,2020.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,1,1,11,2020-08,-,70.5,33.3,2.12,656.2,NoAttempt
180264,CDR,2020.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0,1,0,1,0,0,0,0,1,0,PMG,2020-09,-,14.5,12.8,1.13,1004.7,NoAttempt
180265,CDR,2021.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0,0,0,0,0,1,1,1,1,1,1,2021-04,-,3.6,16.9,0.21,1.6,NoAttempt
180266,CDR,2021.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,0,1,0,0,1,1,1,1,1,1,2021-07,2021-08-05,2.5,5.6,0.45,27.2,Success
180267,CDR,2021.0,4.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0,0,0,0,1,1,0,1,1,1,2021-10,None,0.0,NaN,NaN,NaN,0


In [8]:
stocks.shape

(180268, 31)